# **Joins**

In [ ]:
SELECT 2 + 2;

Write a query that displays all the products along with the SalesOrderID even if an order has never been placed for that product. Display OrderId, ProductId, ProductName and only products which weren’t ordered

In [ ]:
SELECT SalesOrderId, p.ProductID, [Name]
FROM Production.Product AS p
LEFT JOIN Sales.SalesOrderDetail AS o
ON p.ProductID = o.ProductID


Write a query to display count each products within order with product name and order date.


In [ ]:
SELECT p.Name, count(p.ProductId) [Ordered], h.OrderDate
FROM Production.Product AS p
JOIN Sales.SalesOrderDetail AS s
ON p.[ProductID] = s.ProductID
JOIN Sales.SalesOrderHeader AS h
ON s.SalesOrderID = h.SalesOrderID
GROUP BY p.Name, h.OrderDate

Write a query to return list of customer names along with a count of the orders made by these customers.


In [ ]:
SELECT p.FirstName, p.LastName, count(c.CustomerID) [Products ordered]
FROM Sales.Customer AS c 
JOIN Person.Person AS p
ON c.PersonID = p.BusinessEntityID
JOIN Sales.SalesOrderHeader AS s
ON c.CustomerID = s.CustomerID
GROUP BY c.CustomerID, p.FirstName, p.LastName

We have been asked by the Products team for some information about every product and its description for those that have a culture originating in English (En). This information is required so that they know what is coming from England: 

 Task1: identification of tables for required information

 Task2: identify common fields in table to link them together. Write the query that join them together and selects the product identifier, culture, product description, culture,, product model name

 Task3: Filtering for the ‘en’ culture. Adding to the query in task2, retrieve inly products that come from English (en) culture.

In [ ]:
SELECT p.ProductID, m.Name [Model Name ], c.Name [Culture], [Description]
FROM Production.Product AS p
JOIN Production.ProductModel As m
ON p.ProductModelID = m.ProductModelID
JOIN Production.ProductModelProductDescriptionCulture AS cid
ON m.ProductModelID = cid.ProductModelID
JOIN Production.Culture AS c 
ON cid.CultureID = c.CultureID
JOIN Production.ProductDescription AS d
ON cid.ProductDescriptionID = d.ProductDescriptionID
WHERE c.Name = 'English'

We have been asked be Adwenture Work Finance Department to find out the total amount due from two customers Walter Brian and Walter Mays on all order they have placed. It is worth noting each time a customer places an order they are treated as a new customer, so they get a separate record.

 Task1: Investigating the Customers’ data for customers who have first name Walter 

 Task2: Identify whis data should be uset to group and join all appropriate tables. 

 Task3: Use the needed aggregate functions

In [ ]:
SELECT p.FirstName, p.LastName, SUM(h.TotalDue) [Total Due]
FROM Sales.Customer AS s
JOIN Person.Person AS p
ON s.PersonID = p.BusinessEntityID
JOIN Sales.SalesOrderHeader AS h
ON s.CustomerID = h.CustomerID
WHERE p.FirstName like 'Walter' and p.LastName in ('Brian', 'Mays')
GROUP BY p.LastName, p.FirstName

We want to see which employee have been working in more than one department. Use the tables Person. Person, HumanResources.Employee and HumanResources.EmployeeDepartmentHistory. The COUNT function in companion with GROUP BY and HAVING is used to find out the answer to our question.

In [ ]:
SELECT p.FirstName, p.LastName, COUNT(DepartmentID) [N of departments]
FROM HumanResources.Employee AS e
JOIN HumanResources.EmployeeDepartmentHistory AS h
ON e.BusinessEntityID = h.BusinessEntityID
JOIN Person.Person AS p 
ON e.BusinessEntityID = p.BusinessEntityID
GROUP BY e.BusinessEntityID, p.FirstName, p.LastName
HAVING COUNT(DepartmentID) > 1


We want to see information about countries and provinces. Join the CountryRegion and the StateProvince tables (both in the Person schema). Note that we want to keep the countries for which there are no provinces! Sort the result on country name and province name.

In [ ]:
SELECT r.Name [Country Name], p.Name [Province Name]
FROM Person.CountryRegion AS r
JOIN Person.StateProvince AS p
ON r.CountryRegionCode = p.CountryRegionCode
WHERE p.IsOnlyStateProvinceFlag = 1
ORDER BY r.Name, p.Name

# **Set operations**

<span style="font-family: Helvetica; font-size: 11px; color: rgb(0, 0, 0);">Display all record with SalesQuota more than zero (Sales.SalesPerson). Besides add to this result set all historical data with the same condition regarding SalesQuota (display BusinessEntityID, QuotaDate (if there no such column please display current date) and SalesQuoa). Consider that duplicates are acceptable.</span>

In [ ]:
(
    SELECT BusinessEntityID, SalesQuota, GETDATE()
    FROM Sales.SalesPerson
    WHERE SalesQuota > 0
) UNION (
    SELECT BusinessEntityID, SalesQuota, QuotaDate
    FROM Sales.SalesPersonQuotaHistory
    WHERE SalesQuota > 0
)

<span style="font-family: Helvetica; font-size: 11px; color: rgb(0, 0, 0);">Generate a list of unique sutnames from among employes and salespersons</span>

In [ ]:
(
    SELECT p.LastName
    FROM Sales.SalesPerson AS sp
    JOIN Person.Person AS p 
    ON sp.BusinessEntityID = p.BusinessEntityID
) UNION (
    SELECT p.LastName
    FROM HumanResources.Employee AS ep 
    JOIN Person.Person AS p
    ON ep.BusinessEntityID = p.BusinessEntityID
)

<span style="font-family: Helvetica; font-size: 11px; color: rgb(0, 0, 0);">Write a query that returns customer and order ids’ pairs that had order activity in January 2008 but not in February 2008 (use set operators and simplify result’s query with simpler query)</span>

In [ ]:
(
    SELECT CustomerID, SalesOrderID, OrderDate
    FROM Sales.SalesOrderHeader
    WHERE MONTH(OrderDate) = MONTH(CAST('January 2008' AS DATE))
    AND YEAR(OrderDate) = 2008
) EXCEPT (
    SELECT CustomerID, SalesOrderID, OrderDate
    FROM Sales.SalesOrderHeader
    WHERE MONTH(OrderDate) = MONTH(CAST('February 2008' AS DATE))
    AND YEAR(OrderDate) = 2008
)

In [ ]:
SELECT CustomerID, SalesOrderID, OrderDate
FROM Sales.SalesOrderHeader
WHERE MONTH(OrderDate) = MONTH(CAST('January 2008' AS DATE))
AND  MONTH(OrderDate) != MONTH(CAST('February 2008' AS DATE))
AND YEAR(OrderDate) = 2008

<span style="font-family: Helvetica; font-size: 11px; color: rgb(0, 0, 0);">Write a union query involving the EXCEPT operator. Subtract products that are components from the total list of products, leaving only those products that are not components (components are products which are in BillOfMaterials table)</span>

In [ ]:
SELECT ProductID, [Name]
FROM Production.Product
WHERE ProductID in
((
    SELECT ProductID
    FROM Production.Product
) EXCEPT (
    SELECT BillOfMaterialsID
    FROM Production.BillOfMaterials
))

<span style="font-family: Helvetica; font-size: 11px; color: rgb(0, 0, 0);">Write a query to find products that have incurred both good and poor reviews (use Production.ProductReview)</span>

In [ ]:
SELECT [Name], ProductID
FROM Production.Product
WHERE ProductId IN (
(
    SELECT p.ProductID
    FROM Production.Product AS p
    JOIN Production.ProductReview AS r
    ON p.ProductID = r.ProductID
    WHERE Rating >= 4
) INTERSECT (
    SELECT p.ProductID
    FROM Production.Product AS p
    JOIN Production.ProductReview AS r
    ON p.ProductID = r.ProductID
    WHERE Rating <= 3
))

# **Subqueries**

Using a subquery, display the product names and product ID numbers from the Production.Product table that have been ordered


In [ ]:
SELECT [Name], ProductID
FROM Production.Product
WHERE ProductID IN
(
    SELECT ProductID
    FROM Sales.SalesOrderDetail
)

In [ ]:
SELECT DISTINCT [Name], p.ProductID
FROM Production.Product AS p
JOIN Sales.SalesOrderDetail AS s
ON p.ProductID = s.ProductID

Change the query written in question 1 to display the products that have not been ordered. 


In [ ]:
SELECT [Name], ProductID
FROM Production.Product
WHERE ProductID NOT IN
(
    SELECT ProductID
    FROM Sales.SalesOrderDetail
)
ORDER BY ProductID

In [ ]:
SELECT DISTINCT [Name], p.ProductID
FROM Production.Product AS p
JOIN Sales.SalesOrderDetail AS s
-- this one won't work simply by changin !=, thus subquery
-- is optimal here
ON p.ProductID != s.ProductID
ORDER BY ProductID

Write a query that returns all orders placed on the last day of activity that can be found in the SalesOrderHeader table.

TODO: ???

In [125]:
SELECT * 
FROM Sales.SalesOrderHeader
WHERE SalesOrderID in (
    SELECT SalesOrderID
    FROM Sales.SalesOrderHeader
    -- orders places on the last day of activity
    WHERE DueDate = ShipDate
)

(0 rows affected)

Total execution time: 00:00:00.080

SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


Write a query that returns all orders placed by the customer(s) who placed the highest number of orders. Note that more than one customer might have the same number of orders.


In [ ]:
SELECT *
FROM Sales.SalesOrderHeader
WHERE CustomerID in (
    SELECT TOP 1 c.CustomerID
    FROM Sales.Customer AS c
    JOIN Sales.SalesOrderHeader AS h
    ON c.CustomerID = h.CustomerID
    GROUP BY c.CustomerID
    ORDER BY COUNT(h.SalesOrderID) DESC
)